### Write code that calculates 10 day moving average for all tickers.

In [5]:
#!/usr/bin/env python3
import sqlite3
import pandas as pd

# Connect to database and load data
conn = sqlite3.connect('historical_data.db')
query = "SELECT Date, Ticker, Adj_Close FROM stock_data WHERE Adj_Close IS NOT NULL"
df = pd.read_sql_query(query, conn)
conn.close()

# Ensure Date is a datetime type
df['Date'] = pd.to_datetime(df['Date'])

# Sort by ticker and date
df = df.sort_values(['Ticker', 'Date'])

# Calculate 10-day moving average for each ticker
df['MA_10'] = df.groupby('Ticker')['Adj_Close'].transform(lambda x: x.rolling(window=10, min_periods=1).mean())

# Print sample results
print(df)


            Date Ticker   Adj_Close       MA_10
0     2000-01-03   AAPL    0.840094    0.840094
1     2000-01-04   AAPL    0.769266    0.804680
2     2000-01-05   AAPL    0.780523    0.796628
3     2000-01-06   AAPL    0.712978    0.775715
4     2000-01-07   AAPL    0.746751    0.769922
...          ...    ...         ...         ...
19336 2025-08-14   NVDA  182.020004  180.370000
19337 2025-08-15   NVDA  180.449997  181.042999
19338 2025-08-18   NVDA  182.009995  181.243999
19339 2025-08-19   NVDA  175.639999  180.981999
19340 2025-08-20   NVDA  175.399994  180.579999

[19341 rows x 4 columns]


### Write code that calculates the daily drawdown based on the past 250 days for all tickers.

In [9]:
#!/usr/bin/env python3
import sqlite3
import pandas as pd

# Connect to database and load data
conn = sqlite3.connect('historical_data.db')
query = "SELECT Date, Ticker, Adj_Close FROM stock_data WHERE Adj_Close IS NOT NULL"
df = pd.read_sql_query(query, conn)
conn.close()

# Ensure Date is a datetime type
df['Date'] = pd.to_datetime(df['Date'])

# Sort by ticker and date
df = df.sort_values(['Ticker', 'Date'])

# Calculate 250-day drawdown for each ticker (for every day)
def calc_drawdown(x):
    rolling_peak = x.rolling(window=250, min_periods=1).max()
    return (x - rolling_peak) / rolling_peak

df['Drawdown_250'] = df.groupby('Ticker')['Adj_Close'].transform(calc_drawdown)

# Print full results (for every day, every ticker)
print(df)


            Date Ticker   Adj_Close  Drawdown_250
0     2000-01-03   AAPL    0.840094      0.000000
1     2000-01-04   AAPL    0.769266     -0.084310
2     2000-01-05   AAPL    0.780523     -0.070911
3     2000-01-06   AAPL    0.712978     -0.151312
4     2000-01-07   AAPL    0.746751     -0.111111
...          ...    ...         ...           ...
19336 2025-08-14   NVDA  182.020004     -0.006224
19337 2025-08-15   NVDA  180.449997     -0.014796
19338 2025-08-18   NVDA  182.009995     -0.006279
19339 2025-08-19   NVDA  175.639999     -0.041057
19340 2025-08-20   NVDA  175.399994     -0.042367

[19341 rows x 4 columns]


### Write code that calculates the daily rate of change for each ticker and then computes the 10-day rolling average of the absolute values of the rate of change.

In [13]:
#!/usr/bin/env python3
import sqlite3
import pandas as pd

# Connect to database and load data
conn = sqlite3.connect('historical_data.db')
query = "SELECT Date, Ticker, Adj_Close FROM stock_data WHERE Adj_Close IS NOT NULL"
df = pd.read_sql_query(query, conn)
conn.close()

# Ensure Date is a datetime type
df['Date'] = pd.to_datetime(df['Date'])

# Sort by ticker and date
df = df.sort_values(['Ticker', 'Date'])

# Calculate daily rate of change (ROC)
df['ROC'] = df.groupby('Ticker')['Adj_Close'].pct_change()

# Calculate 10-day average absolute ROC
df['AvgAbsROC_10'] = df.groupby('Ticker')['ROC'].transform(lambda x: x.abs().rolling(window=10, min_periods=1).mean())

# Print final consolidated results
print(df)


            Date Ticker   Adj_Close       ROC  AvgAbsROC_10
0     2000-01-03   AAPL    0.840094       NaN           NaN
1     2000-01-04   AAPL    0.769266 -0.084310      0.084310
2     2000-01-05   AAPL    0.780523  0.014633      0.049472
3     2000-01-06   AAPL    0.712978 -0.086538      0.061827
4     2000-01-07   AAPL    0.746751  0.047369      0.058213
...          ...    ...         ...       ...           ...
19336 2025-08-14   NVDA  182.020004  0.002368      0.011434
19337 2025-08-15   NVDA  180.449997 -0.008625      0.009964
19338 2025-08-18   NVDA  182.009995  0.008645      0.007213
19339 2025-08-19   NVDA  175.639999 -0.034998      0.009746
19340 2025-08-20   NVDA  175.399994 -0.001366      0.009232

[19341 rows x 5 columns]
